<a href="https://colab.research.google.com/github/jaksonvelasquez/cun/blob/main/Consulta_secop_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [43]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files
import  io

# Definir constantes para parámetros
PARAM_OPTIONS = {
    'nombre_entidad': 'text', 'nit_entidad': 'text', 'departamento': 'text', 'ciudad': 'text',
    'localizacion': 'text', 'orden': 'text', 'descripcion_del_proceso': 'text', 'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text', 'fecha_de_firma': 'date', 'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date', 'fecha_de_inicio_de_ejecucion': 'date', 'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text', 'proveedor_adjudicado': 'text', 'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text', 'objeto_del_contrato': 'text'
}

# Función para inicializar la UI
def initialize_ui():
    global num_params, submit_button, chosen_params, text_inputs
    num_params = widgets.IntSlider(value=1, min=1, max=len(PARAM_OPTIONS), step=1, description='Number of parameters:')
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(handle_submit)
    chosen_params = []
    text_inputs = {}
    num_params.observe(update_params, 'value')
    display(num_params)

# Función para manejar la presentación de los widgets
def display_widgets():
    widget_list = [num_params] + [widgets.HBox([param, text_inputs[param.description]]) for param in chosen_params] + [submit_button]
    box = widgets.VBox(widget_list)
    display(box)

# Función para actualizar los widgets basado en el número de parámetros seleccionados
def update_params(change):
    clear_output(wait=True)
    chosen_params.clear()
    text_inputs.clear()
    for i in range(num_params.value):
        param_dropdown = widgets.Dropdown(options=list(PARAM_OPTIONS.keys()), description=f'Parameter {i+1}:')
        param_dropdown.observe(update_text_input, names='value')
        chosen_params.append(param_dropdown)
        text_inputs[param_dropdown.description] = widgets.Text(description='Enter value:')
    display_widgets()

# Función para actualizar las entradas de texto basadas en la selección del dropdown
def update_text_input(change):
    param_dropdown = change.owner
    text_input = text_inputs[param_dropdown.description]
    text_input.description = f'Enter {change.new}:'

# Función para manejar el evento de envío
def handle_submit(b):
    query_params = build_query_params()
    fetch_data(query_params)

# Función para construir los parámetros de consulta
def build_query_params():
    query_parts = []
    for param_dropdown, text_input in zip(chosen_params, text_inputs.values()):
        param_name = param_dropdown.value.replace(" ", "_")
        value = text_input.value.strip()
        if PARAM_OPTIONS[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if PARAM_OPTIONS[param_name] == 'text' and value:
            query_parts.append(f"lower({param_name}) like '%{value.lower()}%'")
        elif value:
            query_parts.append(f"{param_name}='{value}'")
    query_string = " AND ".join(query_parts)
    return {"$where": query_string}

# Función para obtener y descargar datos en formato Excel
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.csv'  # Obtenemos los datos en formato CSV
    response = requests.get(base_url, params=query_params)
    if response.ok:
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))  # Convertimos de CSV a DataFrame
        file_name = 'data.xlsx'
        df.to_excel(file_name, index=False)  # Guardamos el DataFrame como Excel
        files.download(file_name)  # Descargamos el archivo Excel
    else:
        print("Failed to fetch data:", response.status_code)

initialize_ui()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>